In [107]:
import numpy as np
import torch
import matplotlib.pyplot as plt

import gym

from gym import spaces

import players
import envs
from evaluation import generate_history, is_t4t

In [108]:
import ray
from ray import tune
from ray.rllib.examples.env.multi_agent import MultiAgentCartPole
from ray.rllib.examples.models.shared_weights_model import \
    SharedWeightsModel1, SharedWeightsModel2, TF2SharedWeightsModel, \
    TorchSharedWeightsModel
from ray.rllib.models import ModelCatalog
# from ray.rllib.policy import PolicySpec
from ray.rllib.utils.framework import try_import_tf
from ray.rllib.utils.test_utils import check_learning_achieved
from ray.tune.registry import register_env

from ray.rllib.env.multi_agent_env import MultiAgentEnv


import ray
from ray.rllib.agents.ppo import PPOTrainer, PPOTorchPolicy
from ray.rllib.agents.ppo import DEFAULT_CONFIG as DEFAULT_CONFIG_PPO

from ray.rllib.agents.dqn import DQNTrainer, DQNTorchPolicy
from ray.rllib.agents.dqn import  DEFAULT_CONFIG as DEFAULT_CONFIG_DQN


from ray.tune.registry import register_env
from ray.tune.logger import pretty_print

ray.init(ignore_reinit_error=True, log_to_driver=False)

2021-08-10 17:28:56,436	INFO worker.py:745 -- Calling ray.init() again after it has already been called.


In [109]:
register_env('two_agent_MG_env', lambda c: envs.TwoAgentMatrixGameEnv(c))
register_env('MG_t4tTD_env', lambda c: envs.MatrixGameEnv(player2=players.TitForTatThenDefect()))
# register_env('MG_t4t_env', lambda c: envs.MatrixGameEnv(player2=players.TitForTat()))
register_env('MG_t4t_env', lambda c: envs.MatrixGameEnv())

In [110]:
trainer_config_dqn0 = DEFAULT_CONFIG_DQN.copy()
# trainer_config_dqn0['num_workers'] = 3
trainer_config_dqn0['n_step'] = 3
trainer_config_dqn0['noisy'] = True
trainer_config_dqn0['v_min'] = -10.
trainer_config_dqn0['v_max'] = 10.
trainer_config_dqn0['model']['fcnet_hiddens'] = [256,32,8]
trainer_config_dqn0['framework'] = 'torch'

trainer_config_dqn0['env'] = 'MG_t4t_env'


In [111]:
# ray.init(ignore_reinit_error=True)
results = tune.run(
    "DQN",
    stop={"training_iteration":2},
    name='test_pretrain',
    config=trainer_config_dqn0,
#     config={
# #         "num_gpus": 0,
# #         "num_workers": 1,
# #         "lr": tune.grid_search([0.01, 0.001, 0.0001]),
#         "config":trainer_config_ppo
#     },
    verbose=1,
    checkpoint_at_end=True,
)

2021-08-10 17:29:06,949	INFO tune.py:549 -- Total run time: 9.29 seconds (9.04 seconds for the tuning loop).


In [112]:
agent0.get_state()

{'experiment_id': 'b8c51c90ee0642ce8c0ddc8510d0bbf1',
 'iteration': 2,
 'timesteps_total': None,
 'time_total': 7.438932657241821,
 'episodes_total': 20,
 'ray_version': '1.4.1'}

In [8]:
agent0 = DQNTrainer(config=trainer_config_dqn0)

2021-08-10 16:52:13,743	INFO trainer.py:696 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.
2021-08-10 16:52:13,840	WARNING util.py:53 -- Install gputil for GPU system monitoring.


In [9]:
agent0.restore(results.get_last_checkpoint())

2021-08-10 16:52:16,882	INFO trainable.py:377 -- Restored on 192.168.1.21 from checkpoint: /home/peter/ray_results/test_pretrain/DQN_MG_t4t_env_b0c4e_00000_0_2021-08-10_16-51-57/checkpoint_000002/checkpoint-2
2021-08-10 16:52:16,883	INFO trainable.py:385 -- Current state after restoring: {'_iteration': 2, '_timesteps_total': None, '_time_total': 7.438932657241821, '_episodes_total': 20}


In [ ]:
agent0.get_policy().get_weights()

In [ ]:
def train(self, stop_criteria):
    """
    Train an RLlib PPO agent using tune until any of the configured stopping criteria is met.
    :param stop_criteria: Dict with stopping criteria.
        See https://docs.ray.io/en/latest/tune/api_docs/execution.html#tune-run
    :return: Return the path to the saved agent (checkpoint) and tune's ExperimentAnalysis object
        See https://docs.ray.io/en/latest/tune/api_docs/analysis.html#experimentanalysis-tune-experimentanalysis
    """
    analysis = ray.tune.run(ppo.PPOTrainer, config=self.config, local_dir=self.save_dir, stop=stop_criteria,
                            checkpoint_at_end=True)
    # list of lists: one list per checkpoint; each checkpoint list contains 1st the path, 2nd the metric value
    checkpoints = analysis.get_trial_checkpoints_paths(trial=analysis.get_best_trial('episode_reward_mean'),
                                                       metric='episode_reward_mean')
    # retriev the checkpoint path; we only have a single checkpoint, so take the first one
    checkpoint_path = checkpoints[0][0]
    return checkpoint_path, analysis

In [86]:
trainer_config_dqn0 = DEFAULT_CONFIG_DQN.copy()
# trainer_config_dqn0['num_workers'] = 3
trainer_config_dqn0['n_step'] = 3
trainer_config_dqn0['noisy'] = True
trainer_config_dqn0['v_min'] = -10.
trainer_config_dqn0['v_max'] = 10.
trainer_config_dqn0['model']['fcnet_hiddens'] = [256,32,8]
trainer_config_dqn1['framework'] = 'torch'


In [87]:
trainer_config_dqn1 = DEFAULT_CONFIG_DQN.copy()
# trainer_config_dqn1['num_workers'] = 3
trainer_config_dqn1['n_step'] = 3
trainer_config_dqn1['noisy'] = True
trainer_config_dqn1['v_min'] = -10.
trainer_config_dqn1['v_max'] = 10.
trainer_config_dqn1['model']['fcnet_hiddens'] = [256,32,8]
trainer_config_dqn1['framework'] = 'torch'


In [88]:
import copy

In [206]:

def env_creator(_):
    return envs.TwoAgentMatrixGameEnv()
single_env = envs.TwoAgentMatrixGameEnv()
env_name = "two_agent_MG_env"
register_env(env_name, env_creator)


obs_space = single_env.observation_space
act_space = single_env.action_space
num_agents = single_env.num_agents
# DQNTorchPolicy0 = DQNTorchPolicy.copy()

def gen_policy(i):
#     return (PPOTorchPolicy, obs_space, act_space, {})
    if i == 0:
        conf = trainer_config_dqn0
#         return (DQNTorchPolicy0, obs_space, act_space, conf)
#         policy0 = copy.copy(agent0.get_policy())
#         policy0 = DQNTorchPolicy(obs_space, act_space, conf)
        policy0 = DQNTorchPolicy
#         policy0.set_weights(agent0.get_weights()['default_policy'])
#         policy0 = lambda c: policy0
        return (policy0, obs_space, act_space, conf)

    elif i == 1:
        conf = trainer_config_dqn1
        return (DQNTorchPolicy, obs_space, act_space, conf)
    else:
        print('NO CONF!')
    
    return (DQNTorchPolicy, obs_space, act_space, conf)

policy_graphs = {}



for i in range(num_agents):
    policy_graphs['agent-' + str(i)] = gen_policy(i)
def policy_mapping_fn(agent_id):
        return 'agent-' + str(agent_id)

In [207]:
trainer_config_dqn1['model']['fcnet_hiddens'] = [3]
policy0 = DQNTorchPolicy(obs_space, act_space, trainer_config_dqn0)


In [208]:
agent0.get_state()

{'experiment_id': 'b8c51c90ee0642ce8c0ddc8510d0bbf1',
 'iteration': 2,
 'timesteps_total': None,
 'time_total': 7.438932657241821,
 'episodes_total': 20,
 'ray_version': '1.4.1'}

In [210]:
policy_graphs['agent-0']

(ray.rllib.policy.policy_template.DQNTorchPolicy,
 Box(0.0, 1.0, (400,), float32),
 Discrete(2),
 {'num_workers': 0,
  'num_envs_per_worker': 1,
  'create_env_on_driver': False,
  'rollout_fragment_length': 4,
  'batch_mode': 'truncate_episodes',
  'train_batch_size': 32,
  'model': {'_use_default_native_models': False,
   'fcnet_hiddens': [3],
   'fcnet_activation': 'tanh',
   'conv_filters': None,
   'conv_activation': 'relu',
   'post_fcnet_hiddens': [],
   'post_fcnet_activation': 'relu',
   'free_log_std': False,
   'no_final_linear': True,
   'vf_share_layers': True,
   'use_lstm': False,
   'max_seq_len': 20,
   'lstm_cell_size': 256,
   'lstm_use_prev_action': False,
   'lstm_use_prev_reward': False,
   '_time_major': False,
   'use_attention': False,
   'attention_num_transformer_units': 1,
   'attention_dim': 64,
   'attention_num_heads': 1,
   'attention_head_dim': 32,
   'attention_memory_inference': 50,
   'attention_memory_training': 50,
   'attention_position_wise_mlp_di

In [211]:
config={
    "log_level": "WARN",
#     "num_workers": 3,
#     "num_cpus_for_driver": 1,
#     "num_cpus_per_worker": 1,
#     "lr": 5e-3,
#     "model":{"fcnet_hiddens": [1024, 512,256,32,8]},
    "multiagent": {
        "policies": policy_graphs,
        "policy_mapping_fn": policy_mapping_fn,
    },
    "env": "two_agent_MG_env",
    'framework': 'torch'
}

In [ ]:
exp_name = 'TA_TEST2'
exp_dict = {
        'name': exp_name,
        'run_or_experiment': 'DQN',
        "stop": {
            "training_iteration": 200
        },
        'checkpoint_freq': 0,
        'checkpoint_at_end': True,
        "config": config,
}
# ray.init()
tune.run(**exp_dict)

In [114]:
policy0 = DQNTorchPolicy(obs_space, act_space, trainer_config_dqn0)


In [135]:
policy0.set_weights(agent0.get_weights()['default_policy'])